In [1]:
from transformers import AutoTokenizer, AutoModel
from tokenizers import Tokenizer
#tokenizer = AutoTokenizer.from_pretrained('human_gpt2-v1')
#然后我们可以使用from_file() 方法从该文件里重新加载 Tokenizer 对象：
# #然后我们可以使用from_file() 方法从该文件里重新加载 Tokenizer 对象：
new_tokenizer = Tokenizer.from_file("tokenizer8.json")
# #或者下面方法
# from transformers import GPT2TokenizerFast
# tokenizer = GPT2TokenizerFast(tokenizer_object=new_tokenizer)

from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=new_tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)
#tokenizer.pad_token = tokenizer.eos_token


# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})

/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
full_model = AutoModel.from_pretrained('mygpt_unigram8')
#model.config.eos_token_id
# print(model.config.pad_token_id)
#model.config.pad_token_id = model.config.eos_token_id

In [3]:
gena_module_name = full_model.__class__.__module__
print(gena_module_name)

transformers.models.gpt2.modeling_gpt2


In [4]:
import importlib
# available class names:
# - BertModel, BertForPreTraining, BertForMaskedLM, BertForNextSentencePrediction,
# - BertForSequenceClassification, BertForMultipleChoice, BertForTokenClassification,
# - BertForQuestionAnswering
# check https://huggingface.co/docs/transformers/model_doc/bert
myclass = importlib.import_module(gena_module_name)
#dir(myclass)

In [5]:
cls = getattr(importlib.import_module(gena_module_name), 'GPT2ForSequenceClassification')
cls

transformers.models.gpt2.modeling_gpt2.GPT2ForSequenceClassification

In [6]:
model = cls.from_pretrained('mygpt_unigram8', num_labels=2)
#dir(model)
#model.config.eos_token_id
#print(model.config.pad_token_id)
model.config.pad_token_id = model.config.eos_token_id
#model.resize_token_embeddings(len(tokenizer))

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at mygpt_unigram8 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from datasets import load_dataset
# load ~11k samples from promoters prediction dataset
dataset = load_dataset("yurakuratov/example_promoters_2k")['train'].train_test_split(test_size=0.1)

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sequence', 'promoter_presence'],
        num_rows: 10656
    })
    test: Dataset({
        features: ['sequence', 'promoter_presence'],
        num_rows: 1184
    })
})

In [9]:
dataset['train'][0]

{'sequence': 'GTGGTGGCTCACGCCTGTAATCCCAGCACAGTGGGAGGCTAAGGCGGGCAGATCACTCGAGGTGAGGAATTTGAGACCAGCCTGGCCAACATGGTGAAACAGTCACAACGACCTCAGGCTTGTTTTCTTCGCTTGGATCCAGCTGCCCTGCCCCCAGTCACTCCCAGAATTTCAGGGCGCGCCCCGTGGCCCCGCCTCTCGGAGGGGGAGGTTTCAGTGAGCCGAGACTGCGCCATTGCACTCCAGCCTGGGCAACAAGAGCGAAACTCCGTCTCGGAAAAAAAAAAAAAAGAAAAAAACTAAAATATCAGTACTCTTTTCACTTAGTCATCACAAACCCCTTTAATCCCATATTACCCTCTACTATCATGGTAGCCTCACCTTTGACGACAGACTAGCTGTGTGACCCAAGTAAATGACTTCAGTGCCACCACCTGTAGGAAGGGGCTGCTAATAGCATCTGCTTCATACTAACGTGTTTTCAGGATCGATCTGCATATAAACGCTCAATACACGGTAACTGATTGTCACATTCTATCCCAGGACGCTTCCATTTGCGGATCTCATGGCCCGGAGGACCTGGGCCGCCTCACAACTGGGTCTGAGTCTTGCGTGGGTCCTCTATATAGGGTGAGAAGCGTGGCGCTCGGTTCCTGCCTCGGGGAAGTCCTGGCGCAGATGGGCCACGGGGCCGGCGTGGAGCTGGAGGCAGATCCCGCCCTTGCAGGGCTGGGTAAGAGACTGAAACTTGGAGAATGAACTGCAGGCGCTGCACATCTCGGGCAGATTTAAAATTCGGTCAGCCCCGCTCACCGACACAAAGCCCACAACCGCGCCCAATAGGGAATCGGCCCTGCGGCCCTCCTGCCACTCGCCCCCAACTATCACAGTCCACGGCGGCCCGTCTCTACTAAAAATACAAAAATTAGCCGGGCGTGGTGGTGGGCGCTTGTAATCCCGGCTACTCAGGAGGCTGAGGCAGGAGA

In [10]:
print('# base pairs: ', len(dataset['train'][0]['sequence']))

# base pairs:  2000


In [11]:
print('tokens: ', ' '.join(tokenizer.tokenize(dataset['train'][0]['sequence'])))

tokens:  GTGG TGGCTCACGCCT GTAATCCCAGCA CAGTG GGAGGCTAAGGC GGGCAG ATCACTC GAGGTGAGG AATTTGAG ACCAGCCTGGCC AACATGGTGAAA CAGTCACA ACGA CCTCAGGCT TGTTTTCTT CG CTTGGA TCCAGCT GCCCTGCCCCCA GTCACT CCCAGAA TTTCAGG GCGCGCCC CGTGG CCCCGCCTC TCG GAGGGGGAGG TTTCAGT GAGCCGAGA CTG CGCCATTGCACT CCAGCCTGGGCA ACAAGAGCGAAA CTCCGTCTC GG AAAAAAAAAAAA AAGAAAAAAA CTAAAAT ATCAGTA CTC TTTTCACTT AGTCATCA CAAACC CCTTTAA TCCCATA TTACCCT CTACTAT CATGGTAG CCTCACC TTTGAC GACA GACTAGCTG TGTGACCC AAGTAAATGA CTTCAGTG CCACCACC TGTAGGA AGGGGCTGC TAATAG CATCTGCTT CATACTA ACGTGT TTTCAGGAT CGA TCTGCATAT AAAC GCTCA ATACAC GGTAACTG ATTGTCA CATTCTAT CCCAGGAC GCTT CCATTTGC GGATC TCATGGCCC GGAGGA CCTGGGCC GCCTCAC AACTGGG TCTGAGTCT TGCGTG GGTCCTC TATATAG GGTGAGAA GCGTGG CGCTC GGTTCCTG CCTCGGG GAAGT CCTGGC GCAGATG GGCCAC GGGGCC GGCGTGG AGCTG GAGGCAG ATCCC GCCCTT GCAGGGCTGGG TAAGAG ACTGAAA CTTGGAGAA TGAA CTGCAGGC GCTG CACATCTC GGGCAG ATTTAAAA TTCGGT CAGCCCCG CTCACC GA CACAAAG CCCACAA CCGCGCC CAATAG GGAATC GGCCCTGC GGCC CTCCTG CCA

In [12]:
print('# tokens: ', len(tokenizer.tokenize(dataset['train'][100]['sequence'])))

# tokens:  265


In [13]:
def preprocess_labels(example):
  example['label'] = example['promoter_presence']
  return example

dataset = dataset.map(preprocess_labels)

Map: 100%|██████████| 1184/1184 [00:00<00:00, 10401.66 examples/s]


In [14]:
def preprocess_function(examples):
  # just truncate right, but for some tasks symmetric truncation from left and right is more reasonable
  # set max_length to 128 to make experiments faster
  return tokenizer(examples["sequence"], truncation=True, max_length=256) #max_length 128

In [15]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 1184/1184 [00:00<00:00, 4667.25 examples/s]


In [16]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [17]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sequence', 'promoter_presence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10656
    })
    test: Dataset({
        features: ['sequence', 'promoter_presence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1184
    })
})

In [18]:
from transformers import TrainingArguments, Trainer
import numpy as np

#tokenizer.pad_token = tokenizer.eos_token 
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy': (predictions==labels).sum() / len(labels)}

# change training hyperparameters to archive better quality
training_args = TrainingArguments(
    output_dir="test_run",
    learning_rate=1e-4,
    lr_scheduler_type="constant_with_warmup",
    warmup_ratio=0.1,
    optim='adamw_torch',
    weight_decay=0.0,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.669700,0.526868,0.733108
2,0.500200,0.570136,0.710304
3,0.247300,0.852644,0.699324
4,0.113100,0.941890,0.706926


/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/liming/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: Us